In [27]:
import requests
import configparser
import pandas as pd

In [24]:
# Read the API key from api.config
config = configparser.ConfigParser()
config.read('nvd_api.config')

API_KEY = config.get('API', 'key')
BASE_URL = "https://services.nvd.nist.gov/rest/json/cves/2.0"

# start date should be now
# end date should be now - 7 days
start_date = "2021-03-01T00:00:00:000 UTC-00:00"
end_date = "2021-03-08T00:00:00:000 UTC-00:00"

params = {"cveId": "CVE-2021-26855"}
headers = {"apiKey": API_KEY}

# Make the API request
response = requests.get(BASE_URL, headers=headers, params=params, timeout=60)

response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
data = response.json()

In [25]:
data

{'resultsPerPage': 1,
 'startIndex': 0,
 'totalResults': 1,
 'format': 'NVD_CVE',
 'version': '2.0',
 'timestamp': '2024-11-24T00:13:01.710',
 'vulnerabilities': [{'cve': {'id': 'CVE-2021-26855',
    'sourceIdentifier': 'secure@microsoft.com',
    'published': '2021-03-03T00:15:12.103',
    'lastModified': '2024-11-21T05:56:53.977',
    'vulnStatus': 'Modified',
    'descriptions': [{'lang': 'en',
      'value': 'Microsoft Exchange Server Remote Code Execution Vulnerability'},
     {'lang': 'es',
      'value': 'Una Vulnerabilidad de Ejecución de código remota de Microsoft Exchange Server. Este ID de CVE es diferente de CVE-2021-26412, CVE-2021-26854, CVE-2021-26857, CVE-2021-26858, CVE-2021-27065, CVE-2021-27078'}],
    'metrics': {'cvssMetricV31': [{'source': 'secure@microsoft.com',
       'type': 'Secondary',
       'cvssData': {'version': '3.1',
        'vectorString': 'CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:N',
        'baseScore': 9.1,
        'baseSeverity': 'CRITICAL',
    

In [28]:
vulnerabilities = data['vulnerabilities']
parsed_data = []

for vuln in vulnerabilities:
    cve_data = vuln['cve']
    cve_id = cve_data['id']
    published = cve_data['published']
    last_modified = cve_data['lastModified']
    status = cve_data['vulnStatus']
    description = next((desc['value'] for desc in cve_data['descriptions'] if desc['lang'] == 'en'), None)
    
    metrics = cve_data.get('metrics', {}).get('cvssMetricV31', [])
    if metrics:
        cvss = metrics[0]['cvssData']
        base_score = cvss['baseScore']
        base_severity = cvss['baseSeverity']
    else:
        base_score = None
        base_severity = None
    
    parsed_data.append({
        'CVE ID': cve_id,
        'Published Date': published,
        'Last Modified Date': last_modified,
        'Status': status,
        'Description': description,
        'Base Score': base_score,
        'Base Severity': base_severity
    })

# Create DataFrame
df = pd.DataFrame(parsed_data)

# Print DataFrame
print(df)

           CVE ID           Published Date       Last Modified Date    Status  \
0  CVE-2021-26855  2021-03-03T00:15:12.103  2024-11-21T05:56:53.977  Modified   

                                         Description  Base Score Base Severity  
0  Microsoft Exchange Server Remote Code Executio...         9.1      CRITICAL  
